In [1]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
import pandas as pd
import logging
import os
import csv
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

2023-01-24 14:31:37.683593: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
folder_name = 'chenyu_51657'

class data:
    def __init__(self, file_name) -> None:

        self.file_name = file_name.replace('.csv','')
        self.file_path = folder_name+'/'+self.file_name+'.csv'
        self.type = self.file_name.split('_')[0]

        if self.type == 'label':

            self.p = float(self.file_name.split('_')[1])
            self.len = int(self.file_name.split('_')[2])

            with open(self.file_path, mode = 'r') as f:
                self.data = pd.DataFrame([[i[0],list([int(r) for r in i[1]])] for i in csv.reader(f)])
                self.data.columns = ['text', 'labels']
            
        elif self.type == 'mask':

            self.p = 1.0
            self.len = int(self.file_name.split('_')[1])

            with open(self.file_path, mode = 'r') as f:
                self.data = pd.DataFrame([['replace MASK', i[0], i[1]] for i in csv.reader(f)])
                self.data.columns = ["prefix", "input_text", "target_text"]
            
        self.train, self.test = train_test_split(self.data, test_size=0.2, random_state=96)
        del self.data

label_data = data('label_0.2_16.csv')
print(label_data.test)

                   text                                            labels
13571  最近我读了一则新闻，岁少年涂振鹏  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
24382  文化日新月异，各种各样的知识浩如  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
1613   到了具体地品评文章作品，有的话和  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
38180  鸟们有时在天空有时停在树枝上婉转  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
35766  别说我一毛不拔，今天花上一毛钱，  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
...                 ...                                               ...
25666  做事要不空运控制自己，适可而止才  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10700  张唐就是因为最后的两位皇帝穷奢极  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
21161  蒙住自己的眼睛，并不等于世界蒙住  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
13684  但如今双方对垒已得不发不是你死就  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
34363     在李太太面前，李先生一向。  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

[8165 rows x 2 columns]


In [15]:
# Optional model configuration
model_args = MultiLabelClassificationArgs(num_train_epochs=2)

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "bert",
    "bert-base-chinese",
    num_labels=label_data.len,
    args=model_args,
)

# Train the model
model.train_model(label_data.train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    label_data.test
)



Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not initialized from the model checkpo

  0%|          | 0/32658 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_0_2


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/4083 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/4083 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/8165 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_0_2


Running Evaluation:   0%|          | 0/1021 [00:00<?, ?it/s]

In [16]:
print(result)

{'LRAP': 0.7638000413888286, 'eval_loss': 0.11213704605425373}
